In [1]:
# enviroment set-up
import os
import sys
from dotenv import load_dotenv
#load_dotenv(override=True)
from os.path import join

sys.path.append("..")
sys.path.append("/home/sanonymous/sato")
sys.path.append("/home/sanonymous/sato/model")


os.environ['BASEPATH'] = '/home/sanonymous/sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/anonymousmlc-ds12-v2/code/Users/svenanonymous/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type_public_bi'


from model import datasets
from sklearn.preprocessing import LabelEncoder
from utils import get_valid_types
import copy
import torch
from torch.utils.data import ConcatDataset

In [2]:
label_enc = LabelEncoder()
label_enc.fit(get_valid_types(os.environ["TYPENAME"]))

LabelEncoder()

In [3]:
label_enc.inverse_transform([0])

array(['AB'], dtype='<U14')

# Loading data in train_sherlock

In [5]:
import json
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from model import models_sherlock
from utils import name2dic

topic = "num-directstr_thr-0_tn-400"

if topic:
    topic_dim = int(name2dic(topic)['tn'])
else:
    topic_dim = None

corpus = "public_bi_num"
percent = 5
test_data_size = 20.0
random_state = 2

column_level_split_file_path = f"/home/sanonymous/semantic_data_lake/data/extract/out/labeled_unlabeled_test_split/{corpus}_{percent}_absolute_{test_data_size}_{random_state}.json"
#embclus_gen_train_data_path = f"D:\\semantic_data_lake\\semantic_data_lake\\emb_clus\\knn_classifier\\out\\gen_training_data\\{corpus}_gen_training_data_1_{distance_threshold}_{percent}_absolute_{test_data_size}.csv"
embclus_gen_train_data_path = f"/home/sanonymous/semantic_data_lake/labeling_functions/combined_LFs/gen_training_data/{corpus}_gen_training_data_all_combined_maj_{percent}_absolute_{test_data_size}_{random_state}.csv"
pretrained_shelock_path = f"sherlock_retrain_embclus_all_combined_maj_labeled{percent}_unlabeledAbsolute_test{test_data_size}_{random_state}.pt"
                
with open(column_level_split_file_path) as f:
    labeled_unlabeled_test_split_file = json.load(f)

#embclus_gen_training_data_path = "D:\\semantic_data_lake\\semantic_data_lake\\emb_clus\\knn_classifier\\out\\gen_training_data\\gen_training_data_1_1e-06_10_70_20.csv"
df_embclus_training_data = pd.read_csv(embclus_gen_train_data_path)
df_embclus_training_data["field_name"] = label_enc.transform(df_embclus_training_data["predicted_semantic_type"].tolist())
#df_embclus_training_data


label_enc = LabelEncoder()
label_enc.fit(get_valid_types(os.environ["TYPENAME"]))

whole_corpus = datasets.TableFeatures("public_bi_num",['char', 'rest', 'par', 'word'], topic_feature=topic, label_enc=label_enc, id_filter=None, max_col_count=100)

train = copy.copy(whole_corpus)

train = train.set_filter_for_column_level_split(["MLB_1+column_4", "MLB_1+column_0", "MLB_1+column_30", "MLB_1+column_35", "MLB_1+column_40"])

train.df_header

/home/sanonymous/sato/tmp/public_bi_num_type_public_bi_header_valid.pkl pickle file found, loading...
public_bi_num_type_public_bi_header_valid Load complete. Time 0.0005321502685546875
Total data preparation time: 0.17621922492980957
Total time for splitting on column-level: 0.0019125938415527344


,field_list,field_names
table_id,,
MLB+MLB_1,"[0, 4, 30, 35, 40]","[0, 1, 13, 56, 93]"


# Loading data in train_CRF_LC

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from model import models_sherlock
from utils import name2dic

topic = "num-directstr_thr-0_tn-400"

if topic:
    topic_dim = int(name2dic(topic)['tn'])
else:
    topic_dim = None

# load single column model
pre_trained_sherlock_loc = join(
    os.environ['BASEPATH'], 'model', 'pre_trained_sherlock', os.environ["TYPENAME"])
classifier = models_sherlock.build_sherlock(['char', 'rest', 'par', 'word'], num_classes=len(
    get_valid_types(os.environ["TYPENAME"])), topic_dim=topic_dim, dropout_ratio=0.35).to(device)

classifier.load_state_dict(torch.load(
            join(pre_trained_sherlock_loc, "all_None.pt"), map_location=device))

In [ ]:
label_enc = LabelEncoder()
label_enc.fit(get_valid_types(os.environ["TYPENAME"]))

whole_corpus = datasets.TableFeatures("public_bi_benchmark",['char', 'rest', 'par', 'word'], topic_feature=topic, label_enc=label_enc, id_filter=None, max_col_count=100)

In [ ]:
train_list = []
train = copy.copy(whole_corpus).set_filter_for_column_level_split(labeled_unlabeled_test_split_file["labeled10"]+labeled_unlabeled_test_split_file["unlabeled70"])
#train = copy.copy(whole_corpus).set_filter_for_column_level_split(['CityMaxCapita_1+column_1', "CityMaxCapita_1+column_2"])

train_list.append(train)

training_data = ConcatDataset(train_list)

In [ ]:
#train.df_header.loc["CityMaxCapita+CityMaxCapita_1"]
train.df_header
df_header = copy.copy(train.df_header)
#df_header

In [ ]:
train = train.overwrite_col_labels_from_embclus(df_embclus_training_data)

In [ ]:
df_header.loc["CityMaxCapita+CityMaxCapita_1"]

In [ ]:
train.df_header.loc["CityMaxCapita+CityMaxCapita_1"]

In [ ]:
for index, row in df_embclus_training_data.iterrows():
    # if index > 2:
    #     break
    #print(index)
    dataset_id = row["table"]
    locator = dataset_id.split("_")[0]
    table_id = locator+"+"+dataset_id
    column_id = int(row["column"].split("_")[1])
    
    field_list = eval(df_header.loc[table_id]["field_list"])
    field_names = eval(df_header.loc[table_id]["field_names"])

    # selection in df_header
    #print(df_header.loc[table_id])
    #print(field_list.index(column_id))
    searched_index = field_list.index(column_id)

    # override field_names with the predicted semantic type
    field_names[searched_index] = row["field_name"]

    #print(field_names)

    df_header.loc[table_id]["field_names"] = str(field_names)

In [ ]:
df_header.loc["CityMaxCapita+CityMaxCapita_1"]

In [ ]:
train_list = []
train = copy.copy(whole_corpus).set_filter([
    "CommonGovernment+CommonGovernment_10"
])

test = copy.copy(whole_corpus).set_filter([
    #"MLB+MLB_1",
    "MLB+MLB_46"
])

train_list.append(train)

training_data = ConcatDataset(train_list)

In [ ]:
batch_size = 1
n_worker = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
training = datasets.generate_batches(training_data,
                                                 batch_size=batch_size,
                                                 shuffle=False,
                                                 drop_last=True,
                                                 device=device,
                                                 n_workers=n_worker)

from tqdm import tqdm
import numpy as np
import numpy.ma as ma
from sklearn.metrics import classification_report
from torchcrf import CRF
y_pred, y_true = [], []

classifier.eval()

training_iter = tqdm(training, desc="Training")
for table_batch, label_batch, mask_batch in training_iter:
    for f_g in table_batch:
        table_batch[f_g] = table_batch[f_g].view(batch_size * 100, -1)

    pred_scores = classifier.predict(table_batch)
    #print(pred_scores[0])
    pred = torch.argmax(pred_scores, dim=1).cpu().numpy()
    #print(pred)

    labels = label_batch.view(-1).cpu().numpy()
    masks = mask_batch.view(-1).cpu().numpy()
    invert_masks = np.invert(masks == 1)
    #print(labels)

    y_pred.extend(ma.array(pred, mask=invert_masks).compressed())
    y_true.extend(ma.array(labels, mask=invert_masks).compressed())

    print("Predictions: ",y_pred)
    print("True types: ",y_true)
    #print(classification_report(y_true, y_pred))

    emissions = classifier(table_batch).view(
                    batch_size, 100, -1).to(device)
    #print("Emissions: ",emissions)

    model = CRF(len(get_valid_types(os.environ["TYPENAME"])), batch_first=True).to(device)
    # load pretrained model
    model_loc = join(os.environ['BASEPATH'], 'model',
                        'pre_trained_CRF', os.environ["TYPENAME"])
    loaded_params = torch.load(
        join(model_loc, "CRF+LDA_pathL.pt"), map_location=device)
    model.load_state_dict(loaded_params['CRF_model'])
    model.eval()

    pred = model.decode(emissions, mask_batch)
    print("SATO Prediction:")
    print(pred)

    print(classification_report(y_true, pred[0]))


In [ ]:
import numpy.ma as ma
labels = label_batch.view(-1).cpu().numpy()
masks = mask_batch.view(-1).cpu().numpy()
invert_masks = np.invert(masks == 1)

ma.array(labels, mask=invert_masks).compressed()

In [ ]:
mask_batch[0][0] = 1
mask_batch[0][1] = 1
mask_batch[0][2] = 0
mask_batch[0][3] = 0
mask_batch[0][4] = 0
print(mask_batch)
print(mask_batch[0].all())
print(mask_batch[:,1])

In [ ]:
%run test_loading_dataset.py

# look in table topic extraction

In [ ]:
from extract.helpers.read_raw_data import get_filtered_dfs_by_corpus
from extract.helpers.utils import valid_header_iter_gen


corpus = "public_bi_benchmark"
TYPENAME = "type78"
header_name = "{}_{}_header_valid.csv".format(corpus, TYPENAME)
header_iter = valid_header_iter_gen(header_name)

print(header_iter)
#raw_df_iter = get_filtered_dfs_by_corpus["publicbibenchmark"](header_iter)

for df_features in header_iter:
    print(df_features)

In [ ]:
import pandas as pd
import ast
import numpy as np
column_level_id_filter = [
    "MLB_1+column_37",
    "MLB_1+column_45", 
    "MLB_46+column_52",
    "MLB_46+column_58"]

df_dic = {}

for training_set in column_level_id_filter:
    dataset_id = training_set.split("+")[0]
    # this locator is only suitable for public_bi corpus
    locator = dataset_id.split("_")[0]
    table_id = locator+"+"+dataset_id
    column_id = int(training_set.split("+")[1].split("_")[1])
    if table_id not in df_dic.keys():
        df_dic[table_id] = [column_id]
    elif table_id in df_dic.keys():
        df_dic[table_id].append(column_id)

# generate new df_header
df_header_new = pd.DataFrame(
    {"table_id": [], "field_list": [], "field_names": []})
## manipulate data_dic
new_data_dic = {}
for f_g in old_data_dic.keys():
    new_data_dic[f_g] = {}
old_data_dic = whole_corpus.data_dic
for table_id in df_dic.keys():
    print(table_id)
    # sort every field_list
    df_dic[table_id] = sorted(df_dic[table_id])

    old_field_list = np.array(eval(
        whole_corpus.df_header.loc[table_id]["field_list"]))
    print(old_field_list)

    old_field_names = np.array(eval(
        whole_corpus.df_header.loc[table_id]["field_names"]))

    new_field_list = df_dic[table_id]
    mask_fields = np.array([x in new_field_list for x in old_field_list])
    new_field_names = old_field_names[mask_fields].tolist()

    df_header_new = df_header_new.append(pd.DataFrame({"table_id": [table_id], "field_list": [
                                            str(new_field_list)], "field_names": [str(new_field_names)]}), ignore_index=True)

    
    for f_g in old_data_dic.keys():
        new_data_dic[f_g][table_id] = old_data_dic[f_g][table_id][np.argwhere(mask_fields == True).flatten()]
    
    
df_header_new = df_header_new.set_index("table_id")


In [ ]:
new_data_dic["rest"]

# Building NN with different number of types

In [ ]:
from models_sherlock import FeatureEncoder, SherlockClassifier, build_sherlock
import torch
from os.path import join

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sherlock_feature_groups = ['char', 'rest', 'par', 'word']


classifier = build_sherlock(sherlock_feature_groups, num_classes=78, topic_dim=400, dropout_ratio=0.35).to(device)

model_loc = join(os.environ['BASEPATH'],
                         'model', 'pre_trained_sherlock', "type78")
pretrained_shelock_path = f"sherlock_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}_255types.pt"

classifier.load_state_dict(torch.load(join(model_loc,pretrained_shelock_path), map_location=device))
print(classifier.linear3)

In [ ]:
print(classifier.linear3.weight.data)

In [ ]:
classifier.linear3.out_features = 255
print(classifier.linear3)
print(classifier.linear3.weight.data)
print(len(classifier.linear3.weight.data))

In [ ]:
import torch.nn as nn

classifier.linear3 = nn.Linear(500, 255)
print(classifier.linear3.weight.data)
print(len(classifier.linear3.weight.data))

In [ ]:
# retrain sherlock and check the las layer 
import numpy as np
import os

# set env-var
os.environ['BASEPATH'] = 'D:\\20120321_anonymous_AZUREML\\sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/anonymousmlc-ds12-v2/code/Users/svenanonymous/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type78'

percent = 50
# if index != 0:
#     continue
comment = f"labeled{percent}_unlabeled{100-20-percent}_test{20}_255types"
column_level_split_file_path = f"D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_{percent}_{100-20-percent}_20.json"
pretrained_shelock_path = f"sherlock_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}_255types.pt"
pretrained_CRF_LDA_path = f"CRF+LDA_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}.pt"

# retrain sherlock
%run ../model/train_sherlock.py -c ../model/params/publicbi/sherlock_retrain.txt  --comment {comment} --column_level_split_file_path {column_level_split_file_path}

In [ ]:
pretrained_shelock_path = f"sherlock_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}_255types.pt"

%run ../model/train_sherlock.py -c ../model/params/publicbi/sherlock_eval.txt --model_list {pretrained_shelock_path} --comment "sherlock_baseline_column_level_split_test20" --column_level_split_file_path "D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_10_70_20.json"
